# Download Bacteriophages genomes from FTP site with Accession from NCBI Virus

In [1]:
## IMPORT ##
import pandas as pd
from pandas import read_csv
import urllib.request
import time
import os
from datetime import date
from pyfaidx import Fasta

Firstly, download RefSeq Genomes from NCBI Virus as csv format from https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Bacteriophage,%20all%20taxids and then read the dataframe.

In [2]:
seq = pd.read_csv('sequences.csv') #sequences from NCBI Virus -> assemblies

Filtr genomes and create list of genomes you want download

In [3]:
#new dataframe with individual columns
new_seq = seq[['Assembly', 'Species', 'Molecule_type', 'Family', 'Host', 'GenBank_Title']].copy()

#sorting values by Host and Family
new_seq.sort_values(by=['Host', 'Family'])

#searching Family Siphoviridae which Host is Lactococcus lactis
siphoviridae = new_seq.loc [(new_seq['Family'] == 'Siphoviridae') & (new_seq['Host'] == 'Lactococcus lactis')] 

#Family Siphoviridae|Lactococcus Lactis Host assemblies -> make list 
asb = siphoviridae["Assembly"].tolist()

In [6]:
siphovir_loc = seq.loc[seq['Family'] == 'Siphoviridae']
sipho_list = siphovir_loc["Assembly"].tolist() #make a list of bacteriophages of interest

In [67]:
sipho_list[0:25]

['GCF_009685705.1',
 'GCF_002620465.1',
 'GCF_002620665.1',
 'GCF_002620365.1',
 'GCF_002633045.1',
 'GCF_003423365.1',
 'GCF_002602865.1',
 'GCF_002602885.1',
 'GCF_002594645.1',
 'GCF_002602825.1',
 'GCF_009828335.1',
 'GCF_008704725.1',
 'GCF_003308595.1',
 'GCF_003308695.1',
 'GCF_003308655.1',
 'GCF_003308755.1',
 'GCF_002619805.1',
 'GCF_009931875.1',
 'GCF_009909565.1',
 'GCF_009662755.1',
 'GCF_011067295.1',
 'GCF_009662515.1',
 'GCF_014183395.1',
 'GCF_014183385.1',
 'GCF_013306735.1']

In [68]:
len(sipho_list)

1579

In [65]:
asb[0:25]

['GCF_009685705.1',
 'GCF_002620465.1',
 'GCF_002620665.1',
 'GCF_002620365.1',
 'GCF_009685685.1',
 'GCF_002611385.1',
 'GCF_002611285.1',
 'GCF_002758415.1',
 'GCF_009662695.1',
 'GCF_002620505.1',
 'GCF_002620585.1',
 'GCF_002602185.1',
 'GCF_002602245.1',
 'GCF_002602225.1',
 'GCF_002602205.1',
 'GCF_002630605.1',
 'GCF_002630585.1',
 'GCF_001746015.1',
 'GCF_001745855.1',
 'GCF_001745355.1',
 'GCF_001745195.1',
 'GCF_001744695.1',
 'GCF_001744535.1',
 'GCF_001744035.1',
 'GCF_001744015.1']

Download assembly_summary_refseq txt file to get the path from specific assemblies

In [32]:
urllib.request.urlretrieve("https://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt", "assembly_summary_refseq.txt")

('assembly_summary_refseq.txt', <http.client.HTTPMessage at 0x7f73d7b71d30>)

In [8]:
def get_assemblies(phages_list, path):
    '''
    This function download genomes from flirting list of Bacteriophages to concrete path
    '''
    #import assembly_summary_refseq file to dataframe
    assembly_sum = pd.read_csv('/home/amanda/assembly_summary_refseq.txt', sep='\t', skiprows=1) 
    #names of columns
    assembly_sum.columns = [
        'assembly_accession',
        'bioproject','biosample',
        'wgs_master','refseq_category',
        'taxid','species_taxid','organism_name',
        'infraspecific_name','isolate','version_status',
        'assembly_level','release_type','genome_rep',
        'seq_rel_date','asm_name','submitter','gbrs_paired_asm',
        'paired_asm_comp','ftp_path','excluded_from_refseq',
        'relation_to_type_material','asm_not_live_date'
    ]

    for assembly in phages_list:
        # searching specific genomes from list
        my_df = assembly_sum[(assembly_sum['assembly_accession'] == assembly)]
        #Process the newly created file and download genomes from NCBI website
        ftp = my_df['ftp_path'].tolist() #making ftp list --> path to download genomes
        asm = my_df['asm_name'].to_list() #making asm list --> asm necessary as part of suffix

        for elem in ftp:
            for i in asm:
                file_in = assembly + '.fna.gz' #gzip format
                fullfilename = os.path.join(path, file_in) #directory and file name
                file_suffix=elem+'/'+assembly+'_'+i+'_genomic.fna.gz'
                try:   
                    if os.path.isfile(fullfilename): #if genome is in directory, skip it and continue the rest of them
                        print(file_in, " already exist")
                        continue
                    else:
                        response = urllib.request.urlretrieve(file_suffix, fullfilename) #download genomes
                        print("Download:", file_in)
                        time.sleep(1)

                except:        
                    print("Skipping", file_in, " - doesn't exist.") #If there is an error or the ftp server doesn't have the genome, skip it


In [19]:
def folder(parent_dir, folder_name):
    '''
    This function create new folder in directory
    '''
    path = os.path.join(parent_dir, folder_name)
    os.mkdir(path)
    print("Directory '% s' created" % folder_name)
    

In [20]:
folder('/home/amanda/Bacteriophages','Siphoviridae')

Directory 'Siphoviridae' created


In [22]:
path = '/home/amanda/Bacteriophages/Siphoviridae' 

In [23]:
get_assemblies(sipho_list, path)

Download: GCF_009685705.1.fna.gz
Download: GCF_002620465.1.fna.gz
Download: GCF_002620665.1.fna.gz
Download: GCF_002620365.1.fna.gz
Download: GCF_002633045.1.fna.gz
Download: GCF_003423365.1.fna.gz
Download: GCF_002602865.1.fna.gz
Download: GCF_002602885.1.fna.gz
Download: GCF_002594645.1.fna.gz
Download: GCF_002602825.1.fna.gz
Download: GCF_009828335.1.fna.gz
Download: GCF_008704725.1.fna.gz
Download: GCF_003308595.1.fna.gz
Download: GCF_003308695.1.fna.gz
Download: GCF_003308655.1.fna.gz
Download: GCF_003308755.1.fna.gz
Download: GCF_002619805.1.fna.gz
Download: GCF_009931875.1.fna.gz
Download: GCF_009909565.1.fna.gz
Download: GCF_009662755.1.fna.gz
Download: GCF_011067295.1.fna.gz
Download: GCF_009662515.1.fna.gz
Download: GCF_014183395.1.fna.gz
Download: GCF_014183385.1.fna.gz
Download: GCF_013306735.1.fna.gz
Download: GCF_016759575.1.fna.gz
Download: GCF_016759545.1.fna.gz
Download: GCF_014190375.1.fna.gz
Download: GCF_015709985.1.fna.gz
Download: GCF_002615565.1.fna.gz
Download: 

In [24]:
def inventory(path):
    '''
    This function create inventory txt file with date of downloading

    '''
    current_time = str(date.today())
    with open(os.path.join(path, "inventory.txt"), "w") as f:
        for path, subdirs, files in os.walk(path):
            for filenames in files:
                if filenames == 'inventory.txt':
                    continue
                else:
                    f.write(filenames + '\t' + current_time + '\n')

            


### Create one file for all downloading genomes with samtools

Change the directory to create one compressed fasta file and *.fai file by which samtools can quickly access any region of the genome.

In [63]:
os.chdir(path)
print("Current Working Directory: " , os.getcwd())

Current Working Directory:  /home/amanda/Bacteriophages/Siphoviridae


In [28]:
%%bash
gunzip GCF*.fna.gz
cat GCF*.fna>> genomy.fasta
rm GCF*.fna
bgzip -c genomy.fasta >> genomy.fasta.bgzf
samtools faidx genomy.fasta

In [29]:
inventory(path)

In [55]:
%%bash
samtools faidx genomy.fasta.bgzf NC_001706.1:1-1234

>NC_001706.1:1-1234
AAGCCTAACTCTTTTGAATCTATTTTTCCATTTTTCGTTTTGAAAAAAATATGCCGTAAT
TCTTCCTAAATCGTCTTATTTTGCGTGTTTTGGTCTGTTCCTTGTCATTGTCCTAGAAAT
TATTTAAAATGCAATACAAGCCAATCTGTGAGCTTAAAGGGGTATTATATAGCAGGGCGT
ACTTCATCAACAACCAAACAAAAAAACCACTCTTGATTGAGTGGCGTGTAAATGAAGAGA
TACCTCACTTTTCTATTTTATTAAATTTTGTTCATTGATAGTTTCAATTGCTTTTTTAAC
TTGTTCAGGTTTACCAGTTATAACTAACTTTATTTCATCTTTACGTTCGGCGTGTTTCTT
TCCTACTTTAATTAAGTACCAACTGTACAAGATGAATGACACAATATAAATAACGTATAC
TATTTTTACACCTCCGTCATTTCTTCTAACGTTTCTTCTACTGTTTTATGTAAGTCGTAG
TAGAATACACCAGTATAATGTTTGTCTTCTTCTTTGGTCCAATTTTCATAATCGGTATCT
TCCATGATTTCGTTGACCATTTCAGAATGTTCGTTTAGGTTAGTCACAAGAATTTCAACG
GCTTTTACAGAGGCTTTGTGCCATTTTGCCGTGAATTCGTACGCTTTGACAACTTCTTGA
AGCATACTGAACAAGTCAATGTATTGGGCTTCAGCATAAGCGTGGACTTGACTTTCATCT
TTTGGAAAATGTTCATCTACTTTTTTATCATGTAATTTCAAAGTGTCACTAAGCAATTCA
ATTTGGTTTTTTAATTTCATTTTTTTATTTCTCTCTTTCTTTTTACACTTGCATTCCAGT
TAATTTGTTTAAGTATTTCGTTTTGCGGTCGATATGATACTCTAAATTGTTTCCCCAACG
CGTTTGTAGCGAAAGTTTTAAACATTCAATAATATAGCTTTTAAGTGTTCCGTTTGTATT
AACA


**For each row in *.fai:**

- Column 1: Accession

- Column 2: The number of bases in the genome

- Column 3: The byte index of the file where the genome sequence begins. (Notice how it constantly increases by roughly the amount in column 2?)

- Column 4: bases per line in the FASTA file

- Column 5: bytes per line in the FASTA file


In [57]:
%%bash

awk 'OFS="\t" {print $1,$2,$3, $4, $5}' genomy.fasta.bgzf.fai

NC_002166.1	40751	50	80	81
NC_002214.1	39807	41388	80	81
NC_002656.1	50550	81748	80	81
NC_002671.1	15179	132988	80	81
NC_000872.1	40739	148413	80	81
NC_002661.2	42942	189720	80	81
NC_003288.1	43081	233257	80	81
NC_003387.1	52797	276931	80	81
NC_001902.1	26111	330447	80	81
NC_001706.1	22172	356936	80	81
NC_001884.1	134416	379437	80	81
NC_001835.1	28451	515586	80	81
NC_002669.1	14957	544451	80	81
NC_002747.1	37667	559651	80	81
NC_003157.5	43106	597955	80	81
NC_001909.1	31754	641655	80	81
NC_002666.1	35538	673864	80	81
NC_002703.1	38347	709903	80	81
NC_002486.1	45636	748792	80	81
NC_002670.1	14510	795057	80	81
NC_003524.1	33507	809805	80	81
NC_000871.1	37370	843787	80	81
NC_002185.1	35466	881680	80	81
NC_001901.1	46375	917638	80	81
NC_003291.2	37618	964640	80	81
NC_001416.1	48502	1002787	80	81
NC_004302.1	33350	1052008	80	81
NC_004681.1	75931	1085830	80	81
NC_004689.1	70797	1162770	80	81
NC_002628.3	31007	1234627	80	81
NC_005091.2	57455	1266082	80	81
NC_003309.1	53373	1324313	80	81
NC_004

In [60]:
%%bash
cut -f1-2 genomy.fasta.bgzf.fai #only the number of bases in genomes

NC_002166.1	40751
NC_002214.1	39807
NC_002656.1	50550
NC_002671.1	15179
NC_000872.1	40739
NC_002661.2	42942
NC_003288.1	43081
NC_003387.1	52797
NC_001902.1	26111
NC_001706.1	22172
NC_001884.1	134416
NC_001835.1	28451
NC_002669.1	14957
NC_002747.1	37667
NC_003157.5	43106
NC_001909.1	31754
NC_002666.1	35538
NC_002703.1	38347
NC_002486.1	45636
NC_002670.1	14510
NC_003524.1	33507
NC_000871.1	37370
NC_002185.1	35466
NC_001901.1	46375
NC_003291.2	37618
NC_001416.1	48502
NC_004302.1	33350
NC_004681.1	75931
NC_004689.1	70797
NC_002628.3	31007
NC_005091.2	57455
NC_003309.1	53373
NC_004686.2	56275
NC_005356.1	41708
NC_004167.1	39325
NC_004303.1	43075
NC_004615.1	43604
NC_004683.1	57050
NC_004682.2	50913
NC_004305.1	42259
NC_004066.1	36798
NC_001900.1	49136
NC_004617.1	42722
NC_004688.1	110865
NC_004902.1	44373
NC_004684.1	67480
NC_005178.1	37611
NC_002668.1	36949
NC_005355.1	40190
NC_005885.1	58638
NC_004616.1	44970
NC_004685.1	69777
NC_004664.2	41831
NC_004746.1	36596
NC_004813.1	57930
NC_00499

### Pyfaidx to get access to any subsequence from file

more about pyfaidx:
https://pypi.org/project/pyfaidx/

In [43]:
genes = Fasta('/home/amanda/Bacteriophages/Siphoviridae/genomy.fasta')

In [44]:
genes.keys()

odict_keys(['NC_002166.1', 'NC_002214.1', 'NC_002656.1', 'NC_002671.1', 'NC_000872.1', 'NC_002661.2', 'NC_003288.1', 'NC_003387.1', 'NC_001902.1', 'NC_001706.1', 'NC_001884.1', 'NC_001835.1', 'NC_002669.1', 'NC_002747.1', 'NC_003157.5', 'NC_001909.1', 'NC_002666.1', 'NC_002703.1', 'NC_002486.1', 'NC_002670.1', 'NC_003524.1', 'NC_000871.1', 'NC_002185.1', 'NC_001901.1', 'NC_003291.2', 'NC_001416.1', 'NC_004302.1', 'NC_004681.1', 'NC_004689.1', 'NC_002628.3', 'NC_005091.2', 'NC_003309.1', 'NC_004686.2', 'NC_005356.1', 'NC_004167.1', 'NC_004303.1', 'NC_004615.1', 'NC_004683.1', 'NC_004682.2', 'NC_004305.1', 'NC_004066.1', 'NC_001900.1', 'NC_004617.1', 'NC_004688.1', 'NC_004902.1', 'NC_004684.1', 'NC_005178.1', 'NC_002668.1', 'NC_005355.1', 'NC_005885.1', 'NC_004616.1', 'NC_004685.1', 'NC_004664.2', 'NC_004746.1', 'NC_004813.1', 'NC_004996.1', 'NC_005354.1', 'NC_005822.1', 'NC_002321.1', 'NC_005345.2', 'NC_006548.1', 'NC_002484.2', 'NC_002667.1', 'NC_006356.2', 'NC_004680.1', 'NC_005284.1'

In [45]:
genes['NC_013153.1'][1:2341]

>NC_013153.1:2-2341
CCGGCCGCGCCCCGCCGCGGGCCGCAAAAAGGCGCTTTTTGCCGCAAAATCTCGCAAAATCTCGCAAAATCTCGCCAAATCTCGCCAAATCTCGCCAAATCTCGCTAAAACTTTGCTAAATTTTAATTATAGGGATATATACTAATATTTTGGCTTGTGAGAGCAGAGTCACCCAAGCGGGTAAACTTATAAGCAAGAGTTATTGCTGCTCTTAGAAACGAAAATATAGGCTTTAAACGATATAAGCACATTAGAAAGGATAATATGCAAACACAAAACGGTGGAAGACCCACAATTTTACCTAAGATGTACGAAGAACCGCTTTTTAGTCAAATCATTGATAAAATTGAATCAGGCTGCAATGACAGAGAAATCTACACCAGTTTGCATTGTTCGGCTAAAACTTTTAGAAAGTGGCGAGATGACAATATAAAGGCGTATGACGAAGCTAAAGGTATCGCTAGGGGAAATCTATTAGAACTAGCTGAAAGTGCCTTAGCAAGCAAACTGACAGTCAGAACGCTAAAAGAAACAGAAACAATCTATGACGCTGACGGAAACGTTGAAAAAGTAAAGGTTAAAGAAAAAGAGCTGGATAAAGATAGCTTAGTAGCAATGATGGTTGCTAAGGCTGGTAACCCTGAACTTTATAACCCTACTGAATGGCGGAGATTGCAACAGGAAGAATCAAGCGCTCATGACCTTAAGGCTAAAATTGAAGAACTTGACGACTATAAACTAAGTAAGTACGAAACACCAGAAATTAAAGCACCAGAGGGATTTGAATGAAACAATATTGGATAATTGAAGACCATTTAGACGGAGGGCTTTATATAATGCCAGAAGATACTCCAGAAGATGAATTAGAAGAAGTTGAAGATACTTGTGCTGAGTGTGGAGACTTTGATTCAGTTATTGGTAAATTTTCAAACTGGAATCAACTAAAAAAAGAAATGACTGATGTTAAAGATTGGTGTCCA

In [46]:
for records in genes:
    print(records.long_name)

NC_002166.1 Bacteriophage HK022, complete genome
NC_002214.1 Streptococcus thermophilus bacteriophage Sfi11, complete genome
NC_002656.1 Mycobacterium phage Bxb1, complete genome
NC_002671.1 Lactococcus prophage bIL312, complete genome
NC_000872.1 Streptococcus phage Sfi21, complete genome
NC_002661.2 Staphylococcus phage phiSLT, complete genome
NC_003288.1 Staphylococcus phage phiETA, complete genome
NC_003387.1 Mycobacterium phage TM4, complete genome
NC_001902.1 Methanobacterium phage psiM2, complete genome
NC_001706.1 Lactococcus phage c2, complete genome
NC_001884.1 Bacillus phage SPBc2, complete genome
NC_001835.1 Lactococcus phage sk1, complete genome
NC_002669.1 Lactococcus prophage bIL310, complete genome
NC_002747.1 Lactococcus phage TP901-1, complete genome
NC_003157.5 Streptococcus pyogenes strain NIH1 putative methionine sulfoxide reductase gene, complete cds; and integrated temperate phage PhiNIH1.1, complete genome
NC_001909.1 Lactococcus phage bIL170, complete genome
NC

In [62]:
for line in genes['NC_002166.1']:
    print(line)

...
AAAACCTTTGCCCTAAATGACCGCCGCCAAA


In [61]:
len(genes['NC_002166.1'])

40751